# Tables

- [ ] Literate review results collation
- [ ] System overview
- [ ] Carbon intensity estimates
- [ ] EPF accuracy metrics
- [ ] MOE and CO2 results

<br>

### Imports

In [26]:
import pandas as pd

from moepy import eda

<br>

### Systems Overview

In [ ]:
# \begin{table}[h]
# \captionof{table}{Systems overview for 2019}\label{table_2}
# \begin{tabular}{|l|l|l|}
# \hline
# Country                             & Germany & Great Britain \\ \hline
# Average Solar/Wind Generation (\%)  & 31.7    & 24.7          \\ \hline
# Average Demand (MW)                 & 60.3    & 32            \\ \hline
# Average Price (EUR/MWH)             & 37.7    & 42.1          \\ \hline
# Average Carbon Intensity  (gCO\textsubscript{2}/kWh) & 338.9   & 187.3         \\ \hline
# \end{tabular}
# \end{table}

In [28]:
df_DE = eda.load_DE_df('../data/energy_charts.csv', '../data/ENTSOE_DE_price.csv')

df_DE.head()

,Biomass,Brown Coal,Gas,Hard Coal,Hydro Power,Oil,Others,Pumped Storage,Seasonal Storage,Solar,Uranium,Wind,net_balance,demand,price
local_datetime,,,,,,,,,,,,,,,
2010-01-03 23:00:00+00:00,3.637,16.533,4.726,10.078,2.331,0.000,0.0,0.052,0.068,0.0,16.826,0.635,-1.229,53.657,NaN
2010-01-04 00:00:00+00:00,3.637,16.544,4.856,8.816,2.293,0.000,0.0,0.038,0.003,0.0,16.841,0.528,-1.593,51.963,NaN
2010-01-04 01:00:00+00:00,3.637,16.368,5.275,7.954,2.299,0.000,0.0,0.032,0.000,0.0,16.846,0.616,-1.378,51.649,NaN
2010-01-04 02:00:00+00:00,3.637,15.837,5.354,7.681,2.299,0.000,0.0,0.027,0.000,0.0,16.699,0.630,-1.624,50.540,NaN
2010-01-04 03:00:00+00:00,3.637,15.452,5.918,7.498,2.301,0.003,0.0,0.020,0.000,0.0,16.635,0.713,-0.731,51.446,NaN


In [29]:
s_DE_RES_output = df_DE[['Wind', 'Solar']].sum(axis=1)
s_DE_demand = df_DE['demand']
s_DE_price = df_DE['price']

s_DE_RES_pct = s_DE_RES_output/s_DE_demand

DE_2020_RES_pct = s_DE_RES_pct['2020'].mean()
DE_2020_demand_avg = s_DE_demand['2020'].mean()
s_DE_price['2020'].mean()

DE_2020_RES_pct, DE_2020_demand_avg

(0.3593124152992342, 55.956133452868855)

In [30]:
DE_fuel_to_co2_intensity = {
    'Biomass': 0.39, 
    'Brown Coal': 0.36, 
    'Gas': 0.23, 
    'Hard Coal': 0.34, 
    'Hydro Power': 0, 
    'Oil': 0.28,
    'Others': 0, 
    'Pumped Storage': 0, 
    'Seasonal Storage': 0, 
    'Solar': 0, 
    'Uranium': 0,
    'Wind': 0, 
    'net_balance': 0 
}

s_DE_emissions_tonnes = (df_DE
                         [DE_fuel_to_co2_intensity.keys()]
                         .multiply(1e3) # converting to MWh
                         .multiply(DE_fuel_to_co2_intensity.values())
                         .sum(axis=1)
                        )

s_DE_emissions_tonnes = s_DE_emissions_tonnes[s_DE_emissions_tonnes>2000]
s_DE_carbon_intensity = s_DE_emissions_tonnes/s_DE_demand.loc[s_DE_emissions_tonnes.index]

DE_2020_emissions_tonnes = s_DE_emissions_tonnes.mean()
DE_2020_ci_avg = s_DE_carbon_intensity.mean()

DE_2020_emissions_tonnes, DE_2020_ci_avg

(11870.320551662837, 201.96611772496715)

In [23]:
# Loading in
df_EI = pd.read_csv('../data/electric_insights.csv')

df_EI = df_EI.set_index('local_datetime')
df_EI.index = pd.to_datetime(df_EI.index, utc=True)

# Extracting fuels
fuel_cols = ['nuclear', 'biomass', 'coal', 'gas', 'hydro', 'pumped_storage', 'solar', 'wind', 'belgian', 'dutch', 'french', 'ireland', 'northern_ireland']
s_ = df_EI[fuel_cols]

# Extracting price
s_price_GB = df_EI['day_ahead_price']

df_fuels_GB.head()

,nuclear,biomass,coal,gas,hydro,pumped_storage,solar,wind,belgian,dutch,french,ireland,northern_ireland
local_datetime,,,,,,,,,,,,,
2009-01-01 00:00:00+00:00,6.973,0.0,17.650,11.900,0.246,-0.404,0.0,0.148,0.0,0.0,1.977,0.0,0.0
2009-01-01 00:30:00+00:00,6.968,0.0,17.770,12.031,0.245,-0.527,0.0,0.157,0.0,0.0,1.977,0.0,0.0
2009-01-01 01:00:00+00:00,6.970,0.0,18.070,11.754,0.246,-1.018,0.0,0.147,0.0,0.0,1.977,0.0,0.0
2009-01-01 01:30:00+00:00,6.969,0.0,18.022,11.162,0.246,-1.269,0.0,0.148,0.0,0.0,1.746,0.0,0.0
2009-01-01 02:00:00+00:00,6.960,0.0,17.998,10.812,0.246,-1.566,0.0,0.160,0.0,0.0,1.730,0.0,0.0


In [19]:
s_DE_RES_output = df_fuels_DE[['Wind', 'Solar']].sum(axis=1)
s_DE_demand = df_fuels_DE.sum(axis=1)

s_DE_RES_pct = s_DE_RES_output/s_DE_demand

DE_2019_RES_pct = s_DE_RES_pct['2019'].mean()
DE_2020_RES_pct = s_DE_RES_pct['2020'].mean()

round(100*DE_2019_RES_pct, 2)

(32.52, 35.93)

<br>

### Carbon Intensity Estimates

In [ ]:
{
    'nuclear': 0, 
    'biomass': 0.121, # from EI 
    'coal': 0.921, # DUKES 2018 value
    'gas': 0.377, # DUKES 2018 value (lower than many CCGT estimates, let alone OCGT)
    'hydro': 0, 
    'pumped_storage': 0, 
    'solar': 0,
    'wind': 0,
    'belgian': 0.4,  
    'dutch': 0.474, # from EI 
    'french': 0.053, # from EI 
    'ireland': 0.458, # from EI 
    'northern_ireland': 0.458 # from EI 
}

<br>

### Electricity Price Forecasting Metrics

<br>

### Price and CO2 MOE Results